In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix


In [2]:
users = pd.read_csv("Users.csv")
restaurants = pd.read_csv("Restaurant.csv")
orders = pd.read_csv("Orders.csv")
menu = pd.read_csv("Menu.csv")
food = pd.read_csv("Food.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'Users.csv'

In [ ]:
users.head(5)

In [ ]:
users.info()

In [ ]:
users.describe()

In [ ]:
users.isnull().sum()

In [ ]:
users.shape

In [ ]:
restaurants.head(5)

In [ ]:
restaurants.info()

In [ ]:
restaurants.describe()

In [ ]:
restaurants.isnull().sum()

In [ ]:
restaurants.shape

In [ ]:
orders.head(5)

In [ ]:
orders.info()

In [ ]:
orders.describe()

In [ ]:
orders.isnull().sum()

In [ ]:
orders.shape

In [ ]:
menu.head(5)

In [ ]:
menu.info()

In [ ]:
menu.describe()

In [ ]:
menu.isnull().sum()

In [ ]:
menu.shape

In [ ]:
food.head(5)

In [ ]:
food.info()

In [ ]:
food.describe()

In [ ]:
food.isnull().sum()

In [ ]:
food.shape

# DATA PREPROCESSING

### USERS

In [ ]:
users

In [ ]:
users.columns

In [ ]:
users.dtypes

In [ ]:
users['Age'].unique()

In [ ]:
users['Marital Status'].unique()

In [ ]:
users['Occupation'].unique()

In [ ]:
users['Occupation'].fillna("Unknown", inplace=True)
users

In [ ]:
users['Marital Status'].value_counts()

### RESTAURANT 

In [ ]:
restaurants

In [ ]:
restaurants.columns

In [ ]:
restaurants.dtypes

In [ ]:
restaurants['Rating'].isnull().sum()

In [ ]:
# Replace "--" with NaN and convert to numeric
restaurants['Rating'] = restaurants['Rating'].replace('--', np.nan)


In [ ]:
restaurants['Rating'] = pd.to_numeric(restaurants['Rating'])

In [ ]:
restaurants['Rating'].unique()

In [ ]:
restaurants['Rating'].value_counts()

In [ ]:
restaurants['Rating'].mean()

In [ ]:
nan_count = restaurants['Rating'].isna().sum()
nan_count

In [ ]:
restaurants.shape

In [ ]:
restaurants['Restaurant_id'].unique()

In [ ]:
restaurants

In [ ]:
restaurants['Rating_count'].unique()


In [ ]:
restaurants['Rating_count'].value_counts()

In [ ]:

# Step 2: Convert Rating_count to numeric values
# Create a mapping dictionary for Rating_count
rating_count_map = {
    'Too Few Ratings': 0,
    '20+ ratings': 20,
    '50+ ratings': 50,
    '100+ ratings': 100,
    '500+ ratings': 500,
    '1K+ ratings': 1000,
    '5K+ ratings': 5000,
    '10K+ ratings': 10000
}

# Apply the mapping to create a new numeric column
restaurants['Rating_count_numeric'] = restaurants['Rating_count'].map(rating_count_map)

# Now both Rating and Rating_count_numeric are numeric and can be used for analysis and modeling

In [ ]:
restaurants

In [ ]:
restaurants

### ORDERS

In [ ]:
orders

In [ ]:
orders.dtypes

In [ ]:
orders['order_date'] = pd.to_datetime(orders['order_date'])

In [ ]:
orders.dtypes

In [ ]:
restaurants['Restaurant_id'].value_counts()

In [ ]:

orders = orders.dropna(subset=['Restaurant_id'])

In [ ]:
 
orders['Restaurant_id'] = orders['Restaurant_id'].astype('Int64')  

In [ ]:
orders.dtypes

In [ ]:
orders.shape

In [ ]:
orders = orders[orders['Sales_amount'] > 0]

In [ ]:
orders.shape

In [ ]:
orders.duplicated().sum()


In [ ]:
orders.isnull().sum()

### MENU

In [ ]:
menu

In [ ]:
menu.dtypes

In [ ]:
menu.isnull().sum()

In [ ]:
   menu['Cuisine'].value_counts()
   

In [ ]:
menu['Price'].describe()

In [ ]:
menu = menu[menu['Price'] > 0]
menu.shape

In [ ]:
menu.dtypes

### FOOD

In [ ]:
food

In [ ]:
orders

In [ ]:
users

In [ ]:
restaurants

In [ ]:
food

In [ ]:
menu

In [ ]:
food.dtypes

## MERGING THE TABLES IN ONE COMMAND DF

In [ ]:
food_df = orders.merge(users, on='User_id', how='left') \
           .merge(restaurants, on='Restaurant_id', how='left')
food_df

In [ ]:
food_df2 = menu.merge(food, on='Food_id', how='left')
food_df2

In [ ]:
food_df.info()

In [ ]:
food_df.isnull().sum()

In [ ]:
food_df.duplicated().sum()

# EXPLORATOY DATA ANALYTICS

### Q1 — Is Revenue Growing or Volatile?



In [ ]:
# Monthly revenue trend

food_df.head(5)

In [ ]:
monthly_revenue = food_df.groupby(food_df['order_date'].dt.to_period('M'))['Sales_amount'].sum()
monthly_revenue

In [ ]:
# Convert to DataFrame 
monthly_revenue = monthly_revenue.reset_index() 
monthly_revenue['order_date'] = monthly_revenue['order_date'].dt.to_timestamp()

In [ ]:
monthly_revenue

In [ ]:
print(monthly_revenue.head())

In [ ]:
# Plot 
import matplotlib.pyplot as plt 
plt.figure(figsize=(12,6))
plt.plot(monthly_revenue['order_date'], monthly_revenue['Sales_amount'], marker='o')
plt.title("Monthly Revenue Trend")
plt.xlabel("Month") 
plt.ylabel("Revenue (INR) CR")
plt.grid(True) 
plt.show()

In [ ]:
# Monthly growth %
monthly_revenue['Growth_%'] = monthly_revenue['Sales_amount'].pct_change() * 100

print(monthly_revenue)

In [ ]:
import matplotlib.pyplot as plt 

plt.figure(figsize=(12,6))
plt.bar(monthly_revenue['order_date'], monthly_revenue['Growth_%'], color='skyblue', edgecolor='black')
plt.title("Monthly Growth % in Revenue")
plt.xlabel("Month") 
plt.ylabel("Growth %")
plt.xticks(rotation=45)  # rotate labels for readability
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
# Revenue volatility (std deviation)

revenue_volatility = monthly_revenue['Sales_amount'].std()
print("Revenue Volatility (Std Dev):", revenue_volatility)

In [ ]:
mean_revenue = monthly_revenue['Sales_amount'].mean()
mean_revenue

In [ ]:
#Coefficient of variation
cv = revenue_volatility / mean_revenue
print("Coefficient of Variation:", cv)

## insights 
 ### From The Above Mothnly Revenue Trend/Monthly Growth /Revenue Volatility  We Observe That The :-
     * Highest Revenu Month Was 2018-01 it has nearly THe 5cr Revenue In That Month
     * Lowes Revenu Month Is 2020-06 
     * IT Downwords After The One Highest Jump In Revenu 
     * ( Somthing Is Lacking in the Service thats Why it happens)
     * Revenue Volatility (Std Dev): ~5.38M INR  
      This means that, on average, monthly revenue fluctuates about ₹5.38 million away from the mean.
     * Our Monthly Revenue IS not Stable Somtime it goes +38% to -36% 
     * Monthly Revenue Mean IS 29219978 cr
     * CV ≈ 0.185 (18.5%)  revenue volatility is moderate-to-low relative to the average. While there are sharp dips (like June 2020, -36%), overall        the fluctuations are not extreme compared to the scale of revenue.

## Q2 — Is Revenue Distribution Skewed?

In [ ]:
# Histogram of sales_amount

plt.figure(figsize=(10,6))
plt.hist(food_df['Sales_amount'], bins=50, color='skyblue', edgecolor='black') 
plt.title("Histogram of Sales Amount") 
plt.xlabel("Sales Amount (INR)") 
plt.ylabel("Frequency") 
plt.yscale('log')
plt.show()

In [ ]:
#Boxplot of sales_amount

plt.figure(figsize=(8,5))
plt.boxplot(food_df['Sales_amount'], vert=False)
plt.title("Boxplot of Sales Amount")
plt.xlabel("Sales Amount (INR)")
plt.show()


In [ ]:
# Skewness & kurtosis

skewness = food_df['Sales_amount'].skew()
kurtosis = food_df['Sales_amount'].kurt()

print("Skewness:", skewness)
print("Kurtosis:", kurtosis)


## Insight:

      * Our business is mostly mass-market towards of small orders dominate
      * But there are occasional premium orders that are very large, creating a long tail.
      * Skewness = 18.36 → Very high positive skew.
      * kurtosis = 569.97 → Extremely heavy-tailed.
      * Revenue is heavy-tailed.
      * A small Number OF Orders Are Contribute But it disproportionately to revenue
      * This explains the volatility you saw earlier — spikes and drops are driven by these outliers.

# Q3 — Which Cities Are Statistically Dominant?

In [ ]:
# Mean revenue per order by city

city_mean = food_df.groupby('City')['Sales_amount'].mean().sort_values(ascending=False)
city_mean

In [ ]:
# Median Revenue per Order by City
city_median = food_df.groupby('City')['Sales_amount'].median().sort_values(ascending=False)
city_median

In [ ]:
# Standard Deviation by City
city_std = food_df.groupby('City')['Sales_amount'].std().sort_values(ascending=False)
city_std

In [ ]:
# Boxplot of Revenue by Top 10 Cities

top10_cities = food_df['City'].value_counts().head(10).index
plt.figure(figsize=(12,6))
df_top10 = food_df[food_df['City'].isin(top10_cities)]
df_top10.boxplot(column='Sales_amount', by='City', grid=False)
plt.title("Revenue Distribution by Top 10 Cities")
plt.suptitle("")
plt.xlabel("City")
plt.ylabel("Sales Amount (INR)")
plt.xticks(rotation=45)
plt.show()

# Insights 
  
* Premium Cities (high median, high volatility): Sirsi, Tirupati, Srikakulam.
* Customers spend big, but order values swing wildly.
* These cities can drive spikes in revenue but are harder to forecast.
* Stable Mass-Market Cities (low median, low volatility): Kovilpatti, Orai, Osmanabad.
* Customers spend small amounts consistently.
* Reliable but less impactful on overall revenue.
* Premium cities → focus on retention and smoothing volatility they are not predictable .
* Mass-market cities → scale volume to grow revenue and theri are stable .


# Q4 — What Is the Distribution of Customer Lifetime Value (CLV)?

In [ ]:
# CLV per User
clv = food_df.groupby('User_id')['Sales_amount'].sum().reset_index()
clv.rename(columns={'Sales_amount':'CLV'}, inplace=True)
clv

In [ ]:
# Histogram of CLV :- Shows how CLV is distributed across all customers.

plt.figure(figsize=(10,6))
plt.hist(clv['CLV'], bins=50, color='skyblue', edgecolor='black')
plt.title("Histogram of Customer Lifetime Value (CLV)")
plt.xlabel("CLV (INR)")
plt.ylabel("Frequency")
plt.yscale("log") 
plt.show()

In [ ]:
# Boxplot of CLV
plt.figure(figsize=(8,5))
plt.boxplot(clv['CLV'], vert=False)
plt.title("Boxplot of Customer Lifetime Value (CLV)")
plt.xlabel("CLV (INR)")
plt.show()

In [ ]:
# 80/20 Distribution Curve :- - Shows whether ~20% of customers contribute ~80% of revenue.

clv_sorted = clv.sort_values(by='CLV', ascending=False)
clv_sorted['cum_revenue_pct'] = clv_sorted['CLV'].cumsum() / clv_sorted['CLV'].sum() * 100

# Fix: Convert range to a list or use numpy array division
# Option 1: Using list comprehension
clv_sorted['cum_customers_pct'] = [(i+1) / len(clv_sorted) * 100 for i in range(len(clv_sorted))]

# Alternative fix using numpy:
# import numpy as np
# clv_sorted['cum_customers_pct'] = np.arange(1, len(clv_sorted)+1) / len(clv_sorted) * 100

plt.figure(figsize=(10,6))
plt.plot(clv_sorted['cum_customers_pct'], clv_sorted['cum_revenue_pct'], marker='o')
plt.title("80/20 Distribution Curve (Pareto)")
plt.xlabel("Cumulative % of Customers")
plt.ylabel("Cumulative % of Revenue")
plt.grid(True)
plt.show()




In [ ]:
mean_clv = clv['CLV'].mean()
median_clv = clv['CLV'].median()
variance_clv = clv['CLV'].var()

print("Mean CLV:", mean_clv)
print("Median CLV:", median_clv)
print("CLV Variance:", variance_clv)

# insights 
   * Mean CLV: 12484.74 Average customer contribution 
   * Median CLV: 1921.0 Typical customer contribution .
   * Variance: Spread of CLV values 1744543872.37
   * variance is high → CLV distribution is unequal, risky dependence on top customers
   * a few customers dominate revenue (premium-heavy)
   *  Confirms heavy-tailed distribution — premium customers exist but are rare.
   * The curve shows that ~20% of customers contribute ~80% of revenue.
   * Most customers contribute small CLV.
   * A small group of customers contribute disproportionately high CLV.
   * Risk: Dependence on premium customers makes revenue volatile.
- Decision
   *  Retain and nurture premium customers (loyalty programs, personalized offers).
   *  Grow mass-market CLV by increasing frequency or upselling.



# Q5 — Is Age Correlated with Spending?

In [ ]:
# Correlation Calculation

food_df[['Age','Sales_amount']].corr()

In [ ]:
# Scatterplot with Regression Line
import seaborn as sns


plt.figure(figsize=(10,6))
sns.regplot(x='Age', y='Sales_amount', data=food_df, scatter_kws={'alpha':0.3}, line_kws={'color':'red'})
plt.title("Age vs Spending (Sales Amount)")
plt.xlabel("Age")
plt.ylabel("Sales Amount (INR)")
plt.show()


# Insights
   *  Age does not influence spending in any significant way.
   * Spending behavior is not driven by age.
   * Age segmentation alone would not be useful for targeting high-value customers.
   * Correlation ≈ 0 → Age is not correlated with spending.
   * Scatterplot confirms → flat regression line, no trend.


# Q6 — Do High Ratings Lead to Higher Revenue?

In [ ]:
# Q6 is about testing whether high ratings are statistically linked to higher revenue.
food_df[['Rating_count_numeric','Sales_amount']].corr()

In [ ]:
#Scatterplot with Regression Line

plt.figure(figsize=(10,6))
sns.regplot(x='Rating_count_numeric', y='Sales_amount', data=food_df, scatter_kws={'alpha':0.3}, line_kws={'color':'red'})
plt.title("Ratings vs Revenue")
plt.xlabel("Rating")
plt.ylabel("Sales Amount (INR)")
plt.show()


# Insights
   * Correlation (Rating vs Sales_amount) = -0.001905 :- It means there is no statistical relationship between ratings and revenue.
   * High ratings do NOT lead to higher revenue in your dataset.
   * Customers are not choosing restaurants based on ratings alone.


# Q7 — Order Frequency Distribution

In [ ]:
# Q7 is about Order Frequency Distribution

order_freq = food_df.groupby('User_id')['order_date'].count().reset_index()
order_freq.rename(columns={'order_date':'Order_Count'}, inplace=True)
order_freq

In [ ]:
# Plot Distribution

plt.figure(figsize=(10,6))
plt.hist(order_freq['Order_Count'], bins=50, color='skyblue', edgecolor='black')
plt.title("Order Frequency Distribution")
plt.xlabel("Number of Orders per User")
plt.ylabel("Number of Customers")
plt.yscale("log")  
plt.show()

In [ ]:
# Box plot
plt.figure(figsize=(8,5))
plt.boxplot(order_freq['Order_Count'], vert=False)
plt.title("Boxplot of Order Frequency per User")
plt.xlabel("Order Count")
plt.show()

In [ ]:
# Statistical Insights
mean_orders = order_freq['Order_Count'].mean()
median_orders = order_freq['Order_Count'].median()
variance_orders = order_freq['Order_Count'].var()

print("Mean Orders:", mean_orders)
print("Median Orders:", median_orders)
print("Variance:", variance_orders)

# Insights 
   * The average customer places about 2 orders.
   * The median being 2 means most customers order only once or twice.
   * Most users are in the 1–3 orders range.
   * Very few users go beyond 5+ orders.
   * Compact box around 1–2 orders.
   * Outliers on the right → a handful of loyal heavy users.
-
   * The distribution is long-tailed:
   * Most customers churn quickly after 1–2 orders.
   * A small minority are repeat loyalists who order many times.
   * This pattern is typical in food delivery: Mass of casual/one-time users.
   * Small group of loyal customers who drive disproportionate revenue.

- Key Decision
  * Our customer base is dominated by low-frequency users.
  * Retention strategy is critical: convert 1–2 order customers into repeat buyers.
  * Heavy repeaters are rare but extremely valuable — nurture them with loyalty programs, personalized offers, or subscription models.



# Q8 — Recency vs Revenue Relationship

In [ ]:
# Find last order date per user

last_order = food_df.groupby('User_id')['order_date'].max().reset_index()
last_order.rename(columns={'order_date':'last_order_date'}, inplace=True)



last_order

In [ ]:
# Calculate recency (days since last order compared to dataset max date)
max_date = food_df['order_date'].max()
last_order['Recency_days'] = (max_date - last_order['last_order_date']).dt.days

last_order

In [ ]:
 # Merge with total revenue per user
user_revenue = food_df.groupby('User_id')['Sales_amount'].sum().reset_index()
recency_revenue = pd.merge(last_order[['User_id','Recency_days']], user_revenue, on='User_id')

recency_revenue


In [ ]:
# Correlation
recency_revenue[['Recency_days','Sales_amount']].corr()

In [ ]:
#Scatterplot

plt.figure(figsize=(10,6))
sns.regplot(x='Recency_days', y='Sales_amount', data=recency_revenue,
            scatter_kws={'alpha':0.3}, line_kws={'color':'red'})
plt.title("Recency vs Revenue")
plt.xlabel("Recency (days since last order)")
plt.ylabel("Total Revenue (INR)")
plt.show()


# Insights
  * Correlation = -0.0673 → weak negative link.
  * as recency increases (customers haven’t ordered for longer), their total revenue tends to be lower
  * Users who haven’t ordered recently spend less
  * Most points are clustered at low revenue, with a few outliers.
  * customers who are inactive longer generally contribute less revenue, but the relationship is weak.


# Churn-Focused EDA

In [ ]:
ref_date = food_df['order_date'].max()
user_features = food_df.groupby('User_id').agg(
    total_orders = ('order_date','count'),
    total_revenue = ('Sales_amount','sum'),
    last_order_date = ('order_date','max')
).reset_index()

# Recency (days since last order)
user_features['recency'] = (ref_date - user_features['last_order_date']).dt.days

# Average order value
user_features['avg_order_value'] = user_features['total_revenue'] / user_features['total_orders']
user_features

In [ ]:
cutoff_date = ref_date - pd.Timedelta(days=90)
user_features['churn'] = (user_features['last_order_date'] < cutoff_date).astype(int)
user_features

#  Q9) — Compare Features

In [ ]:
print(user_features.groupby('churn')[['total_orders','total_revenue','recency','avg_order_value']].mean())

# INSIGHTS
    * Active users: avg 2.42 orders, ₹16,414 revenue, recency ~45 days, avg order value ~₹6,716.
    * Churned users: avg 1.83 orders, ₹11,936 revenue, recency ~48 days, avg order value ~₹6,443.
    * Active users order more often and spend more overall. Recency is slightly better for active users.
    * Frequency and total revenue are strong signals of churn risk. Encourage repeat orders with offers.


# Q10) — Statistical Test

In [ ]:
from scipy.stats import ttest_ind

# Example: total_orders
t_orders = ttest_ind(user_features[user_features['churn']==1]['total_orders'],
                     user_features[user_features['churn']==0]['total_orders'])

# Example: total_revenue
t_revenue = ttest_ind(user_features[user_features['churn']==1]['total_revenue'],
                      user_features[user_features['churn']==0]['total_revenue'])

print("Orders T-test:", t_orders)
print("Revenue T-test:", t_revenue)

# Insights
   * Orders T-test: p-value = 0.0 → significant difference.
   * Revenue T-test: p-value ≈ 1.5e-22 → significant difference.
   * The differences in orders and revenue between churned vs active users are statistically real, not random.



# Q11) — Feature Importance Simulation

In [ ]:
corr_matrix = user_features[['total_orders','total_revenue','recency','avg_order_value','churn']].corr()
print(corr_matrix['churn'])

# Insights
   *  Recency: +0.516 (strong positive correlation with churn).
   * Total revenue: -0.235 (negative correlation).
   * Total orders: -0.184 (negative correlation).
   *  Avg order value: -0.280 (negative correlation).

- Higher recency (longer time since last order) strongly predicts churn.
- More orders and higher revenue reduce churn risk.
- Avg order value also matters — churned users tend to have lower values.


# Q12) RFM 

In [ ]:
# Reference date = max order date in dataset
ref_date = food_df['order_date'].max()
ref_date

In [ ]:
rfm = food_df.groupby('User_id').agg(
    Recency_days = ('order_date', lambda x: (ref_date - x.max()).days),
    Frequency = ('order_date', 'count'),
    Monetary = ('Sales_amount', 'sum')
).reset_index()


rfm.rename(columns={'order_date':'Frequency'}, inplace=True)
rfm.rename(columns={'Sales_amount':'Monetary'}, inplace=True)
rfm.rename(columns={'order_date':'Recency_days'}, inplace=True)


rfm

In [ ]:
# RFM scoring
rfm['R_score'] = pd.qcut(rfm['Recency_days'], 5, labels=[5,4,3,2,1]).astype(int)
rfm['F_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)
rfm['M_score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5]).astype(int)

# Combine into total RFM score
rfm['RFM_Score'] = rfm['R_score'] + rfm['F_score'] + rfm['M_score']
rfm

In [ ]:
# Segmenting Customers
def segment(x):
    if x >= 12: return 'Champions'
    elif x >= 9: return 'Loyal Customers'
    elif x >= 6: return 'Potential Loyalists'
    else: return 'At Risk / Churn'

rfm['Segment'] = rfm['RFM_Score'].apply(segment)
rfm

In [ ]:
# Distribution of Segments

segment_counts = rfm['Segment'].value_counts(normalize=True) * 100
print(segment_counts)

In [ ]:
# Count customers in each segment
segment_counts = rfm['Segment'].value_counts()

# Plot pie chart
plt.figure(figsize=(8,8))
plt.pie(segment_counts, labels=segment_counts.index,
        autopct='%1.1f%%', startangle=140, colors=['gold','skyblue','lightgreen','salmon'])
plt.title("Customer Segmentation by RFM")
plt.show()


# Insights 
   * Strong base of Loyal + Champions (~55%)
   * More than half of our customers are engaged, frequent, and valuable.
   * This is a healthy sign
   * Potential Loyalists (~28%)
   * These customers are on the edge of becoming loyal.:- With the right nudges (discounts, personalized offers, subscription models),
     they can be converted into Loyal or Champions
   *  At Risk / Churn (~17%)
   *  A smaller but important group.
   * They are inactive or low-value, and could churn if ignored.
   * Reactivation campaigns are needed here (reminders, win-back offers).

* Decisions to Take
    - Double down on Champions & Loyal Customers
    - Reward them with loyalty programs, VIP perks, or exclusive deals.
    - Keep them engaged so they don’t slip into “At Risk.”
* Convert Potential Loyalists
    - Target them with personalized promotions.
    - Example: “You’ve ordered 3 times — here’s a special offer if you order again this week.”
    - This group is your biggest growth opportunity.
* Re-engage At Risk Customers
    - Send win-back campaigns (discount codes, free delivery offers).
    - Highlight new menu items or seasonal promotions to spark interest.

our  customer base is healthy and balanced — with a strong core of Loyalists and Champions. The biggest opportunity is to convert Potential Loyalists into Loyal/Champions, while re-engaging the 17% At Risk to reduce churn.

# Q13 — What Is the Overall Churn Rate?

### - Churn rate = % of customers who stop ordering (become inactive) over a given period.


In [ ]:
# Define cutoff date (e.g., last 90 days of dataset)
cutoff_date = food_df['order_date'].max() - pd.Timedelta(days=90)
cutoff_date

In [ ]:
# Active customers = those with orders after cutoff
active_customers = food_df[food_df['order_date'] > cutoff_date]['User_id'].nunique()

active_customers

In [ ]:
# Total customers = all unique users
total_customers = food_df['User_id'].nunique()
total_customers

In [ ]:
# Churned customers = total - active

churned_customers = total_customers - active_customers
churned_customers

In [ ]:
# Churn rate
churn_rate = churned_customers / total_customers * 100
print("Churn Rate: {:.2f}%".format(churn_rate))


# Insights 
  - Cutoff date: March 28, 2020 (last 90 days of dataset).
  - Active customers (ordered after cutoff): 9,457
  - Total customers: 77,235
  - Churned customers: 67,778
  - Churn rate: 87.76%

      - Nearly 9 out of 10 customers stopped ordering in the last 90 days of your dataset.
      - This confirms what earlier analysis showed: most customers only order once or twice, then churn.
      - The customer base is dominated by short‑term, one‑time users, with a small core of loyal repeaters.
 
          -  Retention is the biggest challenge.
              - High churn means marketing spend is constantly needed to acquire new users.
              - Without retention, growth is expensive and unstable.
          - Opportunities:
              - Focus on converting Potential Loyalists into Loyal/Champions.
              - Build loyalty programs (points, subscriptions, free delivery for repeaters).
            - Run reactivation campaigns for churned customers (discounts, personalized offers).
          - Benchmark:
              - Food delivery apps often face high churn, but 87% is extreme.
              - A healthy target is to reduce churn below ~50% over time.

- Your churn rate is very high (87.76%), showing that most customers don’t stick around. 
- The decision should be to prioritize retention strategies
-  loyalty rewards, personalized offers, and reactivation campaigns
- rather than relying only on new customer acquisition.




 


# Q14) - Outlier Detection

In [ ]:
Q1 = food_df['Sales_amount'].quantile(0.25)
Q3 = food_df['Sales_amount'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5*IQR
upper_bound = Q3 + 1.5*IQR

outliers = food_df[(food_df['Sales_amount'] < lower_bound) | 
                   (food_df['Sales_amount'] > upper_bound)]
print("Outliers detected:", outliers.shape[0])


In [ ]:
# Boxplot 

plt.figure(figsize=(8,6))
sns.boxplot(x=food_df['Sales_amount'])
plt.title("Outlier Detection in Sales Amount")
plt.show()

# Insights
   * The boxplot clearly shows a long tail of extreme sales amounts, with ~21,988 points lying outside the whiskers — these are the abnormal orders.


# ML MODEL 

In [ ]:
user_features

In [ ]:
food_df.columns

In [ ]:
dfA_subset = food_df[['User_id','Age','Gender']]

In [ ]:
user_features = user_features.merge(dfA_subset, on='User_id', how='left')
user_features

In [ ]:
# Safely drop City if it exists
if 'City' in user_features.columns:
    user_features = user_features.drop(columns=['City'])

In [ ]:
user_features.dtypes

In [ ]:
user_features['churn'].value_counts(normalize=True)

In [ ]:
print(user_features.head())
print(user_features.shape)

In [ ]:
user_features.isnull().sum()

In [ ]:
# One-hot encode Gender
user_features = pd.get_dummies(user_features, columns=['Gender'], drop_first=True)



In [ ]:
X = user_features.drop(columns=['User_id','last_order_date','churn','recency'])
y = user_features['churn']



In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# logistic Regression

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(max_iter=1000, class_weight='balanced')

scores = cross_val_score(log_model, X, y, cv=5, scoring='roc_auc')
print("Logistic Regression CV ROC AUC:", scores.mean())

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

log_model = LogisticRegression(max_iter=1000, class_weight='balanced')
log_model.fit(X_train, y_train)

log_pred = log_model.predict(X_test)
log_prob = log_model.predict_proba(X_test)[:,1]

print("Logistic Regression")
print(classification_report(y_test, log_pred))
print("ROC AUC:", roc_auc_score(y_test, log_prob))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)

rf_pred = rf_model.predict(X_test)
rf_prob = rf_model.predict_proba(X_test)[:,1]

print("Random Forest")
print(classification_report(y_test, rf_pred))
print("ROC AUC:", roc_auc_score(y_test, rf_prob))

In [ ]:
pip install xgboost

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=5, random_state=42, scale_pos_weight=7)
xgb_model.fit(X_train, y_train)

xgb_pred = xgb_model.predict(X_test)
xgb_prob = xgb_model.predict_proba(X_test)[:,1]

print("XGBoost")
print(classification_report(y_test, xgb_pred))
print("ROC AUC:", roc_auc_score(y_test, xgb_prob))

In [ ]:
models = {
    "Logistic": roc_auc_score(y_test, log_prob),
    "RandomForest": roc_auc_score(y_test, rf_prob),
    "XGBoost": roc_auc_score(y_test, xgb_prob)
}
print(models)

In [ ]:
import pandas as pd

rf_importance = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False)
rf_importance.plot(kind='bar', figsize=(10,5))

xgb_importance = pd.Series(xgb_model.feature_importances_, index=X.columns).sort_values(ascending=False)
xgb_importance.plot(kind='bar', figsize=(10,5))

In [ ]:
user_features

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

# Logistic Regression
log_prob = log_model.predict_proba(X_test)[:,1]
fpr_log, tpr_log, _ = roc_curve(y_test, log_prob)
auc_log = roc_auc_score(y_test, log_prob)

# Random Forest
rf_prob = rf_model.predict_proba(X_test)[:,1]
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_prob)
auc_rf = roc_auc_score(y_test, rf_prob)

# XGBoost
xgb_prob = xgb_model.predict_proba(X_test)[:,1]
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, xgb_prob)
auc_xgb = roc_auc_score(y_test, xgb_prob)

# Plot all curves
plt.figure(figsize=(8,6))
plt.plot(fpr_log, tpr_log, label=f"Logistic (AUC = {auc_log:.2f})")
plt.plot(fpr_rf, tpr_rf, label=f"Random Forest (AUC = {auc_rf:.2f})")
plt.plot(fpr_xgb, tpr_xgb, label=f"XGBoost (AUC = {auc_xgb:.2f})")

plt.plot([0,1],[0,1],'k--')  # diagonal line
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison")
plt.legend(loc="lower right")
plt.show()

In [ ]:
import joblib
joblib.dump(rf_model, "random_forest_churn.pkl")